# Gep top20 recommendation - XAI recsys: EBM with only side information
# Ranking data: ranking_data_random

In [1]:
import caffeine
caffeine.on(display=False)

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
# Run this for reproduce
# Gets the current working directory
import os
cwd = os.getcwd()
print("Working directory:", cwd)
# Go up one directory from working directory
os.chdir("..")

Working directory: /Users/anhtth/Library/CloudStorage/OneDrive-UniversityofTwente/2023 UT- THESIS/1-Code/0.cb12_main/nb_recsys_ebm


In [4]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm
import random
import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load random ranking data
ranking_data = pd.read_csv('./nb_recsys_ebm/ranking_data_random.csv')

In [ ]:
# # Load ranking data (generated by KNN-LDA features)
# ranking_data_knn = pd.read_csv('./nb_recsys_ebm/ranking_data_knn_lda_v2.csv')

In [ ]:
# ranking_data_knn.head()

In [ ]:
# rec_users = ranking_data_knn.UserID.unique()

In [ ]:
# rec_jobs = ranking_data_knn.JobID.unique()

In [ ]:
# # Load random ranking data
# ranking_data_random = pd.read_csv('./nb_recsys_ebm/ranking_data_random.csv')
# ranking_data = ranking_data_random[ranking_data_random.UserID.isin(rec_users)]

In [6]:
ranking_data.head()

,UserID,JobID,label,City,State,Country
0,13,821691,1,0.0,1.0,1.0
1,13,701157,0,0.0,0.0,0.0
2,13,472398,0,0.0,0.0,0.0
3,13,411244,0,0.0,0.0,0.0
4,13,868940,0,0.0,0.0,0.0


In [7]:
len(ranking_data)

371600

In [8]:
features_df = pd.read_csv('./nb_recsys_ebm/features_df_lda.csv')

In [9]:
features_df.head()

,UserID,JobID,City,State,Country,DegreeType,CurrentlyEmployed,ManagedOthers,WorkHistoryTopic,WorkHistoryLevel,SeniorLevel,ReqTopic,DescTopic,TitTopic
0,7,309823,1.0,1.0,1.0,1,1,0,0,1,4.0,18,2,0
1,7,703889,1.0,1.0,1.0,1,1,0,0,1,4.0,18,2,0
2,7,566574,0.0,0.0,1.0,1,1,0,0,1,4.0,0,18,0
3,7,481216,0.0,0.0,1.0,1,1,0,0,1,4.0,18,15,0
4,9,809208,1.0,1.0,1.0,1,1,0,0,1,1.0,5,18,0


In [ ]:
# ranking_data[ranking_data.JobID.isin(features_df.JobID.values)==False]

In [10]:
# lda jobs features
features_job = ['ReqTopic', 'DescTopic', 'TitTopic']
# lda & user profile features
features_user = ['DegreeType', 'CurrentlyEmployed', 'ManagedOthers', 'WorkHistoryTopic',
       'WorkHistoryLevel', 'SeniorLevel']
# location features
features_location = ['City', 'State', 'Country']

In [11]:
from topN_ebm import *

In [12]:
from topN_ebm_random import *

In [13]:
# !cat topN_ebm.py

In [14]:
job_fm = pd.read_csv('./data_interim/jobs_fm.csv')
job_fm.head()

,JobID,ReqTopic,DescTopic,TitTopic
0,1,5,15,0
1,4,18,15,0
2,7,15,18,0
3,8,15,7,0
4,9,15,7,0


In [15]:
user_fm = pd.read_csv('./data_interim/users_fm.csv')
user_fm.head()

,UserID,Split,DegreeType,CurrentlyEmployed,ManagedOthers,WorkHistoryTopic,WorkHistoryLevel,SeniorLevel
0,80,Train,1,1,1,9,2,4.0
1,123,Train,4,1,0,0,0,3.0
2,162,Train,5,0,0,0,3,6.0
3,178,Train,1,0,1,0,3,6.0
4,344,Train,1,1,0,0,1,3.0


# Start topN for a given user_id

- features_job_matrix: feature names for job matrix
- features_work_matrix: feature names for work history matrix
- user_profile_features: list of features for user profile

In [16]:
%%time
import gzip, pickle, pickletools

filepath = "./output_ebm/ebm_side_info_dropid.pikle"

with gzip.open(filepath, 'rb') as f:
    p = pickle.Unpickler(f)
    ebm_side_info = p.load()

CPU times: user 17.1 ms, sys: 9.28 ms, total: 26.4 ms
Wall time: 33.7 ms


In [ ]:
# !cat topN_ebm_random.py

In [17]:
%%time
rec_result_ebm_side_info = topN_ebm_side_info(model = ebm_side_info, 
           N = 20, 
           ranking_data = ranking_data, 
           user_fm=user_fm, job_fm=job_fm)

CPU times: user 46min 21s, sys: 1min 19s, total: 47min 41s
Wall time: 39min 26s


In [18]:
# Function: get rec_result corresponding to a user id
# rec_result: overall rec_recsult dictionary (output of topN_flat)
def get_rec_result_df(u_id, rec_result):
    rec_20_df = pd.DataFrame(rec_result[u_id])
    rec_cols = ['JobID', 'Y_prob', 'Y_pred']
    rec_20_df.columns = rec_cols
    rec_20_df['UserID'] = u_id
    rec_20_df['rank'] = rec_20_df.groupby('UserID').cumcount()
    return rec_20_df


In [19]:
user_ids = list(ranking_data.groupby('UserID').UserID.unique().astype('int'))

In [20]:
%%time
final_rec_result = pd.DataFrame(columns = ['JobID', 'Y_prob', 'Y_pred','UserID','rank'])

for u_id in user_ids:
    
    temp_df = get_rec_result_df(u_id=u_id, 
                                rec_result=rec_result_ebm_side_info)
    # print(u_id, len(temp_df))
    final_rec_result = pd.concat([final_rec_result, temp_df])

final_rec_result = final_rec_result[['UserID','JobID', 'Y_prob', 'Y_pred', 'rank']]

CPU times: user 13.3 s, sys: 206 ms, total: 13.5 s
Wall time: 13.8 s


In [21]:
final_rec_result.head()

,UserID,JobID,Y_prob,Y_pred,rank
0,13,900877,0.665981,1,0
1,13,324559,0.661431,1,1
2,13,904793,0.631122,1,2
3,13,1066465,0.631122,1,3
4,13,890398,0.631122,1,4


In [22]:
final_rec_result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 74320 entries, 0 to 19
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   UserID  74320 non-null  object 
 1   JobID   74320 non-null  object 
 2   Y_prob  74320 non-null  float64
 3   Y_pred  74320 non-null  object 
 4   rank    74320 non-null  object 
dtypes: float64(1), object(4)
memory usage: 3.4+ MB


In [23]:
final_rec_result.to_csv('./output_topN_ebm/rec_result_side_info_random.csv', header=True, index=False)